# ミニプロジェクト --- 太宰治か宮沢賢治か？著者を推定してみよう

著者が分からない小説について、太宰治の作品か宮沢賢治の作品かを推定する手法を実装しましょう。  

このプロジェクトでは、[青空文庫](https://www.aozora.gr.jp/index.html) のテキストを使用します。
ただし、ルビや入力者注を削除するなど編集を加えています。 
また、両著者の小説の総文字数がおよそ同じになるように、太宰治の小説数を調整しています。

太宰治の小説が81、宮沢賢治の小説が148、未知の著者の小説が10与えられています。
太宰治と宮沢賢治の文章の傾向をそれぞれ調べて、未知の10の小説それぞれを太宰治と宮沢賢治のどちらが書いたかを推測します。
文章の傾向を評価する指標はいくつか提案されていますが、その中で文字n-gramに対する統計的指標を用います。  

例えば、宮沢賢治の小説では、文末が「です、ます」調であることが多いです。
一方、太宰は「である」調が多いです。
したがって、各著者の小説のbi-gramを集計すると、
宮沢賢治の小説では`です`や`す。`の出現確率が高く、太宰治の小説では`ある`や`る。`の出現確率が高くなります。
未知の著者の小説の`です`や`す。`の出現確率が`ある`や`る。`よりも高ければ、
著者は宮沢賢治ではないか？と予想することができそうです。

ですが、宮沢賢治の小説でも「である」調を使っているものがあります。
そこで、著者ごとに、すべての文字n-gramの出現確率の分布を求めて、
未知の著者の文字n-gram確率分布がどちらの著者のものに近いかを調べることにしましょう。  

# Miniproject --- Guessing the author of a novel, 太宰治 or 宮沢賢治?

Let's implement a method that guesses who wrote a novel of an unknown author, 太宰治 (Osamu Dazai) or 宮沢賢治 (Kenji Miyazawa).

In this project, we use texts from [Aozora Bunko](https://www.aozora.gr.jp/index.html).
We have modified the texts by removing rubies and notes written by those who typed the texts.
We have also adjusted the number of novels by 太宰治 so that
the total numbers of letters in novels of the two authors are nearly the same.

We provide 81 novels by 太宰治, 148 novels by 宮沢賢治, and 10 novels by unknown authors.
Observe the characteristics of the texts by 太治宰 and 宮沢賢治, and for each of the novels by unknown authors,
guess who wrote it, 太宰治 or by 宮沢賢治.
Among the known features for classifying texts, try to use statistical features of letter n-grams.

For example, in the novels by 宮沢賢治, sentences tend to end in the です-ます style.
On the other hand, in the novels by 太宰治, sentences tend to end with the である style.
Therefore, given that the bi-grams in the novels by each author have been counted,
the probability of occurrences of `です` or ` す。` should be high in the novels by 宮沢賢治,
and that of `ある` or `る。` should be high in those by 太宰治.
If the probability of `です` or ` す。` in the novel by an unknown author is higher than that of `ある` or `る。`,
you can then conclude that the author is 宮沢賢治.

However, some novels by 宮沢賢治 are written in the です-ます style.
Therefore, compute the probabiliity distribution of all the letter n-grams of each of the authors,
and observe the similarity between the probability distribution of letter-ngrams of an unknown author
and that of 宮沢賢治 (太宰治).

# 基礎課題

## 準備

第4回本課題で、json形式のファイルに入っている小説のテキストを扱いました。
個々の小説は辞書によって表され、そのような辞書のリストがファイルに格納されています。
このプロジェクトでは、`novels.json` というファイルに入っている辞書のリストを用います。
辞書のリストを大域変数 `novels` に設定します。

# Basic exercises

## Preparation

In the exercise of the 4th lecture, you delt with texts of novels stored in a file in json format.
Each novel is represented by a dictionary, and a list of such dictionaries is stored in the file.
In this project, you use a list of such dictionaries stored in the file `novels.json`.
The global variable `novels` is defined as the list of dictionaries.

In [1]:
import json

with open('novels.json', 'r', encoding='utf-8') as f:
    novels = json.load(f)

第3回本課題で、n-gram の求め方を学びました。
以下の関数 `multiline_ngrams(n,text)` は、与えられたテキスト `text` に含まれるn-gramのリストを返します。

In the exercise of the 3rd lecture, you learned how to compute n-grams.
The following function `multiline_ngrams(n,text)` computes a list of n-grams that appear in the given text `text`.

In [2]:
def multiline_ngrams(n, text):
    l = []
    for sentence in text.split('\n'):
        for i in range(0, len(sentence)-n+1):
            l.append(sentence[i:i+n])
    return l

以下の課題では、次のジェネレータを用いてもよいです。

You can also use the following generator in the following exercises.

In [3]:
def multiline_ngrams_gen(n, text):
    for sentence in text.split('\n'):
        for i in range(0, len(sentence)-n+1):
            yield sentence[i:i+n]

## 課題１：ngramの抽出

引数 `novels` に入っている小説のうち、`author` が書いた小説に現れるすべてのn-gramのリストもしくはイテレータを返す関数
`author_ngrams(n,author,novels)` を定義してください。
引数 `author` に著者名（太宰治か宮沢賢治）が渡されます。
引数 `novels` には、大域変数 `novels` の値が渡されることが想定されます。

各n-gramは、著者のすべての小説に現れる回数だけリストもしくはイテレータに現れなければなりません。

## Exercise 1: Extraction of n-grams

Define a function `author_ngrams(n,author,novels)` which returns a list or iterator of all n-grams
that appear in the texts in `novels` each of which is written by `author`.
the parameter `author` is 太宰治 or 宮沢賢治.
The value of the global variable `novels` is given as the parameter `novels`.

Each n-gram must appear in the list or iterator as many times as it appears in all the novels by the author.

In [4]:
def author_ngrams(n, author, novels): #もっと短く書ける
    novels_author = []
    for i in range(len(novels)):
        if novels[i]["author"] == author:
            novels_author.append(novels[i])
    
    ngrams_list = []
    for i in range(len(novels_author)):
        ngrams_list += multiline_ngrams(n, novels_author[i]["text"])
        
    return ngrams_list

以下の文を実行して `True` のみが表示されることを確認してください。

Execute the following statements and check if only `True` is printed.

In [5]:
print(len(list(author_ngrams(3, '太宰治', novels))) == 899275)
print(len(list(author_ngrams(3, '宮沢賢治', novels))) == 864223)
print(list(author_ngrams(3, '太宰治', novels)).count('である') == 2891)
print(list(author_ngrams(3, '宮沢賢治', novels)).count('である') == 290)

True
True
True
True


未知の著者の名前は `'UN0'` ～ `'UN9'` と定義されています。

The names of unknown authors are denoted as `'UN0'` ... `'UN9'`.

In [6]:
print(len(list(author_ngrams(3, 'UN3', novels))) == 13425)

True


## 課題２：n-gramの出現回数

n-gramのリストもしくはイテレータが与えられたとき、各々のn-gramをキーとして、
その出現回数を値（バリュー）とする辞書を返す関数 `histogram(ngs)` を定義してください。
引数 `ngs` には、n-gramのリストが与えられます。

## Exercise 2: Occurrences of n-grams

Define a function `histogram(ngs)`,
which is given a list or iterator of n-grams and returns a dictonary whose keys are n-grams
with the number of their occurrences as their value.
The parameter `ngs` is a list of n-grams.

In [7]:
def histogram(ngs):
    ngram_count_dic = {}
    for i in range(len(ngs)):
        if ngram_count_dic.get(ngs[i]) == None:
            ngram_count_dic[ngs[i]] = 1
        else:
            ngram_count_dic[ngs[i]] += 1
    return ngram_count_dic

以下の文を実行して `True` のみが表示されることを確認してください。

Execute the following statements and check if only `True` is printed.

In [8]:
dazai_histogram = histogram(author_ngrams(3, '太宰治', novels))
miyazawa_histogram = histogram(author_ngrams(3, '宮沢賢治', novels))
un0_histogram = histogram(author_ngrams(3, 'UN0', novels))
print(len(dazai_histogram) == 268576)
print(len(miyazawa_histogram) == 244874)
print(dazai_histogram['である'] == 2891)
print(miyazawa_histogram['である'] == 290)
print(dazai_histogram['です。'] == 1203)
print(miyazawa_histogram['です。'] == 1866)
print(un0_histogram['である'] == 4)
print(un0_histogram['です。'] == 18)

True
True
True
True
True
True
True
True


## 課題３：n-gramの確率分布

課題２で定義した `histogram` によって算出されたn-gram出現回数の分布 `hist` が与えられたら、
n-gramの確率分布を返す関数 `probability_distribution(hist)` を定義してください。
各n-gramの出現回数を、全n-gramの出現回数の総和で割ればよいです。
関数 `probability_distribution` は、n-gramをキーとしてその出現の確率を値とする辞書を返します。

## Exercies 3: Probability ditributions of n-grams

Define a function `probability_distribution(hist)`,
which is given a distribution of occurrences of n-grams `hist` computed by the function `histogram` in Exercise 2,
and returns the probability distribution of n-grams.
The number of occurrences of each n-gram is divided by the total number of occurrences of all the n-grams.
The function `probability_distribution` returns a dictionary
whose keys are n-grams with the probability of their occurrences as their value.

In [9]:
def probability_distribution(hist):
    value_list = hist.values()
    count_all = 0
    for i in value_list:
        count_all += int(i)
        
    pd_ngram = {}
    for key in hist:
        pd_ngram[key] = int(hist[key])/count_all
    return pd_ngram

以下の文を実行して `True` のみが表示されることを確認してください。

Execute the following statements and check if only `True` is printed.

In [10]:
print(round(probability_distribution(dazai_histogram)['である']*10**8) == 321481)
print(round(probability_distribution(miyazawa_histogram)['である']*10**8) == 33556)

True
True


## 課題４：n-gramの確率分布間の距離

いよいよ異なる文書におけるn-gramの確率分布の間の距離を計算していきます。
$d_1$ と $d_2$ という二つのn-gram確率分布が与えられたとしましょう。
以下の数式では、n-gram $x$ に対して $d_i(x)$ は $d_i$ における $x$ の確率を表します。

$d_1$ と $d_2$ のTankard距離は、各々のn-gramの二つのテキストにおける出現確率の差の総和です。
同じn-gramの確率の差が大きければ大きいほど、二つの文書は異なると考えられます。
すべてのn-gramに対してその差の平均を求めます。
したがって、Tankard距離は次のように定義されます。 

$\mbox{Tankard}(d_1, d_2) = 
\frac{1}{\mbox{card}(C)} \sum_{x \in C} {|d_1(x) - d_2(x)|}$

ここで、$C$ は $d_1$ と $d_1$ の両方で確率が正になるn-gramの集合を表していて、以下のように定義されます。

$C = \{~x~|~d_1(x)>0~\mbox{かつ}~d_2(x) > 0 \}$

$\mbox{card}(C)$ は集合 $C$ の要素数を表します。

n-gram確率分布が辞書で表されているとき、辞書に登録されていないn-gramの確率は0と考えます。

では、二つのn-gram確率分布が辞書 `d1` と `d2` として与えられたとき、それらの間のTankard距離を返す関数 `Tankard(d1,d2)` を定義してください。

## Exercise 4: Distance between probability distributions of n-grams

You now compute the distance between probability distributions of n-grams in different texts.
Suppose that two n-gram probability distributions $d_1$ and $d_2$ are given.
In the following mathematical expressions, $d_i(x)$ denotes the probability of an n-gram $x$ in $d_i$.

The Tankard distance between $d_1$ and $d_2$ is obtained by adding the difference of the probabilities
of each n-gram in the two texts.
If the difference of the probabilities of each n-gram is larger, the two texts are considered more different.
The average difference for all n-grams is then computed.
The Tankard distance is therefore defined as follows.

$\mbox{Tankard}(d_1, d_2) =
\frac{1}{\mbox{card}(C)} \sum_{x \in C} {|d_1(x) - d_2(x)|}$,

in which $C$ denotes the set of n-grams whose probabilities in $d_1$ and $d_1$ are both positive, that is,

$C = \{~x~|~d_1(x)>0~\mbox{ and }~d_2(x) > 0 \}$,

and $\mbox{card}(C)$ denotes the number of elements in $C$.

If an n-gram probability distribution is represented by a dictionary,
the probability of an n-gram that is not stored in the dictionary is considered 0.

Now, define a function `Tankard(d1,d2)` that returns the Tankard distance
between the two n-gram probability distributions `d1` and `d2` that are given as dictionaries.

In [11]:
def Tankard(d1, d2):
    C = []
    d1_key = list(d1.keys())
    d2_key = list(d2.keys())
    if len(d1_key) >= len(d2_key):
        for key in d1_key:
            if d2.get(key) != None: #.countはエラー出やすい。.getが無難
                C.append(key)
    else:
        for key in d2_key:
            if d1.get(key) != None:
                C.append(key)   
    card_C = len(C)
    
    abs_sum = 0
    for c in C:
        abs_sum += abs(d1[c] - d2[c])
    
    tankard = abs_sum / card_C
    return tankard

以下の文を実行して `True` のみが表示されることを確認してください。

Execute the following statement and check if only `True` is printed.

In [12]:
print(round(Tankard(probability_distribution(dazai_histogram),probability_distribution(miyazawa_histogram))*10**8) == 857)
print(Tankard(probability_distribution(dazai_histogram),probability_distribution(miyazawa_histogram)))

True
8.56638847730773e-06


## 課題５：著者の推定

著者が未知の小説のそれぞれに対して、太宰治か宮沢賢治のどちらが書いたかを推測しましょう。
次の関数 `which_author(n,un,novels)` を定義してください。
* 関数 `which_author(n,un,novels)` は、正の整数 `n` と未知の著者 `un`（`'UN0'` ～ `'UN9'`）と小説のリスト `novels` を受け取り、
* 著者 `un` と太宰治のn-gram確率分布の間のTankard距離と、
* 著者 `un` と宮沢賢治の間の距離を求め、
* `un` と太宰治の距離が`un` と宮沢賢治の方よりも小さければ、文字列 `'太宰治'` を返し、
* そうでなければ、文字列 `'宮沢賢治'` を返します。

## Exercise 5: Guessing the author

For the novel by each unknown author, try to guess who wrote it, 太宰治 or 宮沢賢治.
Define a function `which_author(n,un,novels)`.
* The function `which_author(n,un,novels)` is given a positive integer `n`,
the name of an unknown author `un` (`'UN0'` ... `'UN9`), and the list of novels `novels`.
* It then computes the Tankard distance between the n-gram probability distribution of the author `un` and that of 太宰治, and
* the distance between the ditribution of `un` and that of 宮沢賢治.
* If the distance between `un` and 太宰治 is smaller than that between `un` and 宮沢賢治, it returns `'太宰治'`, and
* it returns `'宮沢賢治'` otherwise.

In [13]:
def which_author(n, un, novels):
    tankard_dazai_un = Tankard(probability_distribution(histogram(author_ngrams(n, un, novels))), 
                               probability_distribution(histogram(author_ngrams(n, "太宰治", novels))))
    tankard_miyazawa_un = Tankard(probability_distribution(histogram(author_ngrams(n, un, novels))), 
                               probability_distribution(histogram(author_ngrams(n, "宮沢賢治", novels))))
    if tankard_dazai_un < tankard_miyazawa_un:
        return "太宰治"
    else:
        return "宮沢賢治"

以下の文を実行して `True` のみが表示されることを確認してください。

Execute the following statements and check if only `True` is printed.

In [14]:
print(which_author(3,'UN0',novels) == '太宰治')
print(which_author(3,'UN9',novels) == '宮沢賢治')

True
True


以下の文は `'UN0'` ～ `'UN9'` のすべてを検証します。`True` のみが表示されることを確認してください。

The following statement verifies all of `'UN0'` ... `'UN9`.  Check if only `True` is printed.

In [15]:
with open('novels.json', 'r', encoding='utf-8') as f:
    for novel in json.load(f):
        if 'UN' in novel['author']:
            print(which_author(3, novel['author'], novels) == novel['true_author'])

True
True
True
True
True
True
True
True
True
True


# 発展課題

以上の課題では、既に著者（太宰治か宮沢賢治）がわかっている小説を用いて、未知の著者の小説を誰が書いたかを推測しました。

この課題では、239の小説を、著者の情報を用いずに本文だけを用いて、二つのグループ（クラスタ）に分類することを試みてください。

たとえば、7-2で学んだk-means法 `KMeans` を用いることができます。
そのためには、各小説に対して、何らかの特徴量を定義しなければなりません。
いくつかのn-gramの出現確率を用いることができるでしょう。
* たとえば、bi-gramやtri-gramのうち、出現回数が多いものを200個程度選びます。
* 各小説に対して、それぞれのn-gramの出現回数を小説の長さで割って特徴量とします。
* `KMeans` を呼び出すときには、クラスタ数を2ではなく、3以上にするとうまく行くかもしれません。
* `n_init=100` などと指定して `n_init` を増やすと結果が安定します（デフォールトは10）。

用いたクラスタリングの方法と書いたプログラム、および、クラスタリングの結果を報告してください。
もちろん、完全に正しいクラスタを得ることは非常に難しいです。
自分なりに試したこととその結果（悪くても構いません）を報告ください。

# Advanced exercise

In the above exercises, you tried to guess who wrote novels by unknowns authors,
given a set of novels of known authors (太宰治 or 宮沢賢治),

In this exercise, let's try to classify 239 novels into two groups (clusters),
only using the texts of the novels

For example, the k-means method `KMeans` explained in 7-2 can be used.
For using `KMeans`, you should define some features for each novel.
The probabilities of some n-grams can be used.
* For example, choose about 200 bi-grams or tri-grams which are highly ranked with respect to the number of occurrences.
* For each novel, define a feature as the number of occurrences of an n-gram divided by the length of the novel.
* When calling `KMeans`, the number of clusters need not be 2. Specifying 3 or more clusters may produce a good result.
* If you increase`n_init` as in `n_init=100` (its default is 10), the result will be more stable.

Report the method for clustering you use, the program you write, and the result of clustering.
Of course, to obtain completely correct clusters is very difficult.
Report what you try and the result, which may be not so good.

なお、以下のようにして、各小説に対して、著者の情報を取り出すことができます。
（未知の小説に対しては、`true_author` というキーで本当の著者が格納されています。）

Information about the authors can be extraced from `novels.json` as follows.
(For the novels of unknown authors, the actual authors are stored with the key `true_author`.)

In [16]:
answers = ['太宰治' if novel['author']=='太宰治' or novel.get('true_author',None)=='太宰治' else '宮沢賢治' for novel in novels]
answers

['太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '太宰治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 '宮沢賢治',
 